In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import nengo
import nengolib
import numpy as np
import pandas as pd
import random
import scipy as sp

import pytry

In [2]:
from sklearn.decomposition import PCA

class PatternInterpolationTrial(pytry.Trial):
    def params(self):
        self.param('number of neurons', n_neurons=2000),
        self.param('number of training patterns', n_train_pattern=3)
        
    def evaluate(self, q):
        data = pd.read_csv("pinsoro-2017-06-12-140651300755-small.csv", low_memory=False)
        
        x = np.array(data.iloc[:,11:195]).astype(float)
        a = []
        for i in range(x.shape[1]):
            y = pd.Series(x[:,i])
            z = y.interpolate(limit_direction='both')
            a.append(z)
        a = pd.DataFrame(a)
        a = a.dropna()
        a = np.array(a).T

        dt = 0.001
        
        def extract_pattern(start, end):
            pattern = np.array(a[start:end,:]).astype(float)
            frames = np.array(data.iloc[start:end,9]).astype(int)

            good_indices = frames != -1
            frames = frames[good_indices]
            pattern = pattern[good_indices]

            fps = 30.0
            t_sample = (frames - frames[0])/fps

            t = np.arange(int(t_sample[-1]/dt))*dt

            result = []
            for i in range(pattern.shape[1]):       
                p = np.interp(t, t_sample, pattern[:,i])
                result.append(p)
            result = np.array(result).T

            return t, result


        t1, result1 = extract_pattern(8991, 9357) #goaloriented
        t2, result2 = extract_pattern(13811, 14182) #goaloriented
        t3, result3 = extract_pattern(22753, 22853) #goaloriented
        t4, result4 = extract_pattern(26617, 26983) #goaloriented
        t5, result5 = extract_pattern(30051, 30419) #goaloriented

        t6, result6 = extract_pattern(9796, 10174) #noplay
        t7, result7 = extract_pattern(14583, 14991) #noplay 
        t8, result8 = extract_pattern(22854, 23059) #noplay
        t9, result9 = extract_pattern(23902, 24419) #noplay 
        t10, result10 = extract_pattern(30420, 30595) #noplay

        goal_list = [result1, result2, result3, result4, result5]
        noplay_list = [result6, result7, result8, result9, result10]
        
        n_train_pattern=q.n_train_pattern
        
        goal_list1 = random.sample(goal_list, n_train_pattern)
        noplay_list1 = random.sample(noplay_list, n_train_pattern)
        train_all = np.vstack(goal_list1+noplay_list1)
        pca_model = PCA(n_components=1).fit(train_all)

        goal_patterns = {}
        i=0
        for p in goal_list1:
            goal_patterns[i] = pca_model.transform(p)[:,0]
            i+=1

        noplay_patterns = {}
        for p in noplay_list1:
            noplay_patterns[i] = pca_model.transform(p)[:,0]
            i+=1

        patterns = {}
        patterns.update(goal_patterns)
        patterns.update(noplay_patterns)

        goal_patterns = [(goal_patterns[key]) for key in goal_patterns.keys()]
        noplay_patterns = [(noplay_patterns[key]) for key in noplay_patterns.keys()] 

        theta = 1.0
        net = nengo.Network()
        with net:
            process = nengo.processes.WhiteSignal(period=100., high=1.0, y0=0)
            rw = nengolib.networks.RollingWindow(theta=theta, n_neurons=q.n_neurons, process=process, neuron_type=nengo.LIFRate(tau_rc=0.02, tau_ref=0.002, amplitude=1))
            
        s_window = int(theta/dt)
        s_pattern = len(min(patterns.values(), key=len))


        t_window = np.linspace(0, 1, s_window)
        inv_basis = rw.inverse_basis(t_window)

        eval_points=[]
        target = []
        for i in range(s_pattern):
            for gp in goal_patterns:
                eval_points.append(np.dot(inv_basis, np.roll(gp, i)[:s_window]))
                target.append([1])
            for npp in noplay_patterns:
                eval_points.append(np.dot(inv_basis, np.roll(npp, i)[:s_window]))
                target.append([-1])

        eval_points = np.array(eval_points)
        
        with net:
            result = nengo.Node(None, size_in=1)
            nengo.Connection(rw.state, result,
                             eval_points=eval_points, scale_eval_points=True,
                             function=target, synapse=0.1)

#EXTRACT TEST PATTERNS
        t11, result11 = extract_pattern(23560, 23901) #aimless

        testp1 = pca_model.transform(result11)[:,0]
       
        goalp = goal_patterns[0]
        aimp = testp1
        noplayp = noplay_patterns[0]
        
        p_all = np.hstack([goalp, aimp, noplayp])

        presentation_time_test=30/len(p_all)

        model = nengo.Network()
        model.networks.append(net)
        with model:    
            stim = nengo.Node(nengo.processes.PresentInput(p_all, presentation_time=presentation_time_test))
            nengo.Connection(stim, rw.input, synapse=None)

            p_result = nengo.Probe(result)
            p_stim = nengo.Probe(stim)
        sim = nengo.Simulator(model)
        sim.run(30)

        #plt.plot(sim.trange(), sim.data[p_stim], label='input')
        #plt.plot(sim.trange(), sim.data[p_result], label='output')
        #plt.legend(loc='best')



        goalp_len = int(len(goalp)*presentation_time_test/dt)
        aimp_len = goalp_len+int(len(aimp)*presentation_time_test/dt)
        noplayp_len = aimp_len+int(len(noplayp)*presentation_time_test/dt)

        mean1 = np.mean(sim.data[p_result][0:goalp_len])
        std1 = np.std(sim.data[p_result][0:goalp_len])
        median1 = np.median(sim.data[p_result][0:goalp_len])

        mean2 = np.mean(sim.data[p_result][goalp_len:aimp_len])
        std2 = np.std(sim.data[p_result][goalp_len:aimp_len])
        median2 = np.median(sim.data[p_result][goalp_len:aimp_len])

        mean3 = np.mean(sim.data[p_result][aimp_len:noplayp_len])
        std3 = np.std(sim.data[p_result][aimp_len:noplayp_len])
        median3 = np.median(sim.data[p_result][aimp_len:noplayp_len])

        #print("Goal Oriented Mean:   ",mean1," SD: ",std1," Median: ",median1)
        #print("Aimless Mean:         ",mean2," SD: ",std2," Median: ",median2)
        #print("No Play Mean:         ",mean3," SD: ",std3," Median: ",median3)
        
        
        return dict(
            mean_goal=mean1,
            std_goal=std1,
            median_goal=median1,
            mean_aimless=mean2,
            std_aimless=std2,
            median_aimless=median2,
            mean_noplay=mean3,
            std_noplay=std3,
            median_noplay=median3,
        )


In [3]:
#PatternInterpolationTrial().run(n_train_pattern=2, data_dir='debug')

In [4]:
for seed in range(5):
    for n_neurons in [200, 500, 1000, 2000, 5000]:
        PatternInterpolationTrial().run(seed=seed, n_neurons=n_neurons,
                                        data_dir='vary_neurons')


running PatternInterpolationTrial#20190125-111426-96e2a5ec


C:\Users\mbartlett2\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\signal\filter_design.py:1551: BadCoefficients: Badly conditioned filter coefficients (numerator): the results may be meaningless
  "results may be meaningless", BadCoefficients)
C:\Users\mbartlett2\AppData\Local\Continuum\anaconda3\lib\site-packages\nengo\utils\numpy.py:79: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  v = a[inds]
C:\Users\mbartlett2\AppData\Local\Continuum\anaconda3\lib\site-packages\nengolib\signal\system.py:717: UserWarning: Filtering with non-SISO systems is an experimental feature that may not behave as expected.
  "expected.", UserWarning)


HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

<IPython.core.display.Javascript object>

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

...

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

<IPython.core.display.Javascript object>

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

<IPython.core.display.Javascript object>

seed = 4
n_neurons = 5000
n_train_pattern = 3
mean_aimless = 0.014887425272764765
mean_goal = 0.3234019212381076
mean_noplay = -0.6605750704993912
median_aimless = 0.049240965821930334
median_goal = 0.3378004682674556
median_noplay = -0.7383257281447553
std_aimless = 0.7116893790059897
std_goal = 0.49879092234749123
std_noplay = 0.33168112205344263


In [5]:
for seed in range(5):
    for n_train_pattern in [1,2,3,4,5]:
        PatternInterpolationTrial().run(seed = seed, data_dir='vary_train_pattern', 
                                        n_train_pattern = n_train_pattern)

running PatternInterpolationTrial#20190125-113154-04c09b89


C:\Users\mbartlett2\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\signal\filter_design.py:1551: BadCoefficients: Badly conditioned filter coefficients (numerator): the results may be meaningless
  "results may be meaningless", BadCoefficients)
C:\Users\mbartlett2\AppData\Local\Continuum\anaconda3\lib\site-packages\nengo\utils\numpy.py:79: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  v = a[inds]
C:\Users\mbartlett2\AppData\Local\Continuum\anaconda3\lib\site-packages\nengolib\signal\system.py:717: UserWarning: Filtering with non-SISO systems is an experimental feature that may not behave as expected.
  "expected.", UserWarning)


HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

<IPython.core.display.Javascript object>

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

...

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

<IPython.core.display.Javascript object>

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

<IPython.core.display.Javascript object>

seed = 4
n_neurons = 2000
n_train_pattern = 5
mean_aimless = 0.36139742312445594
mean_goal = -0.3235858808470053
mean_noplay = -0.8190026495743346
median_aimless = 0.46401725701798
median_goal = -0.46467997013067824
median_noplay = -0.9230032941020239
std_aimless = 0.6121106941460003
std_goal = 0.5553464672495961
std_noplay = 0.35408778198195145
